# Estimation of risk measures : Value-at-Risk and Conditional-Value-at-Risk

This notebook covers algorithms proposed in the JMLR paper (url) for computing sample-based estimates of two popular convex risk measures : Utility-based shortfall risk (UBSR) and Optimized Certainty Equivalent (OCE) risk. 

The goal of this exercise is to demonstrate correctness of the algorithms by comparing the solutions of the algorithms with the known, true risk values.

## UBSR Estimation

**Utility-based Shortfall Risk**: Given a random variable $X$, the UBSR of $X$ for the choice of loss function '$l$' and a risk threshold '$\lambda$' is defined as 

$SR_{l,\lambda}(X) = \inf \left\{ t \in \mathrm{R} | \mathbf{E}\left[ l(-X-t) \right] \le \lambda \right\}$

**UBSR Estimation Problem** : Given 'm' samples of $X$ : $\left\{Z_i\right\}_{i=1}^m$, estimate $SR_{l,\lambda}(X)$.

In [1]:
import plotly.io as pio
pio.renderers.default = 'colab'
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib notebook

import warnings
warnings.filterwarnings(action="ignore")

from distributions import Gaussian, Student_t, Laplace, Pareto, Logistic, GEV
from algorithms.utility_based_shortfall_risk import SR
from algorithms.optimized_certainty_equivalent import OCE

SYMBOLS = ['circle','square','star']
PLOTLY_COLORS_DARK = px.colors.qualitative.Dark2
PLOTLY_COLORS_LIGHT = px.colors.qualitative.Set2 

### Distributions
A collection of distributions for which Value-at-Risk (quantile) and Conditional Value-at-Risk (superquantile) are known.

#### Experimental Setup

For every sample size $m \in M$, we repeat the UBSR estimation problem $N$ times. For each $m$, we plot error mean and variance computed across these $N$ simulations. 

In [2]:
M, N = [10,100,1000], 100
Alpha = np.linspace(0,1,26,endpoint=False)[1:]
delta, epsilon = 1e-3, 1

#### Example 1: Value-at-Risk

- **Intuitively**: Suppose the r.v. $L$ denotes loss,i.e., higher positive values of $L$ imply more losses. Value-at-Risk of $L$ at level $\alpha \in [0,1]$ is the maximum possible loss after excluding the worst losses with a combined probability of atmost $\alpha$.
- **Mathematically**: Suppose the r.v. $X$ denotes gain, i.e., higher positive values of $X$ imply more gain. Then,

  $$VaR_{\alpha}(X) = - \inf\left\{ x \in \mathbf{R} | Pr(X \leq x)>\alpha\right\}$$.

  Thus $VaR_{\alpha}(X)$ is equal to the $(1-\alpha)$-quantile of $-X$.
- For the above implemented distributions of $X$, following holds: $(1-\alpha)$-quantile of $-X$ equals the **negative** of ($\alpha$-quantile of $X$).
- $SR_{l,\lambda}(X)$ coincides with $VaR_\alpha(X)$ measure for the choice of loss function : $l(x) = 1_{\{x>0\}}$, and threshold $\lambda=\alpha$
- We consider distributions with continuous and strictly monotone CDF, for e.g., Gaussian, Exponential, Student, Pareto and Logistic distributions.
- For above distributions, the parametric form for the quantile function (and therefore VaR) is available.
- We use 25 values for $\alpha$, spread uniformly between $0$ and $1$, and we repeat the experiment for each choice of quantile $\alpha$.

In [3]:
def simulate_VaR(l, delta, Alpha, M, distribution):
    UBSR = [SR(l,a) for a in Alpha]
    true_SR = -distribution.quantile(Alpha)
    Error_Means, Error_Stds = [], []
    for m in M:
        error_mean, error_std = [], []
        for i, ubsr in enumerate(UBSR):
            errors = []
            for _ in range(N):
                Z = distribution.sample(m)
                t = ubsr.UBSR_SB(Z,delta/np.sqrt(m))
                errors.append(t - true_SR[i])
            errors_np = np.array(errors)
            error_mean.append(np.abs(errors_np).mean())
            error_std.append(errors_np.std())
        Error_Means.append(error_mean)
        Error_Stds.append(error_std)
    return np.array(Error_Means), np.array(Error_Stds)

In [4]:
l = lambda x : np.where(x > 0, 1, 0)

In [8]:
distributions = [
    Gaussian(5,10), Pareto(4),
    Student_t(3, 3, 5), Student_t(5, -3, 4),
    Logistic(0,1), Laplace(1,1),
    GEV(1,1,0.5), GEV(1,1,-0.5)
]
subplot_titles = ['Gaussian ', 'Pareto ',
                 "Student's t (df = 3)", "Student's t (df = 5)",
                 'Logistic', 'Laplace',
                 'GEV (Frechet)', 'GEV (reversed Weibull)']

fig = make_subplots(rows=4, cols=2, subplot_titles = subplot_titles, vertical_spacing=0.1, horizontal_spacing=0.1)
rows, cols = [1,1,2,2,3,3,4,4], [1,2] * 4
labels = ['m='+str(m) for m in M]
for row, col, distribution in zip(rows, cols, distributions): 
    Error_Means, Error_Stds = simulate_VaR(l, delta, Alpha, M, distribution)
    flag = (row == 1) and (col == 1)
    for i in range(len(M)):
        fig.add_trace(
            go.Scatter(x=Alpha, y=Error_Means[i], line=dict(color=PLOTLY_COLORS_DARK[i]), showlegend=flag, name = labels[i], mode='markers', marker=dict(symbol=SYMBOLS[i], size=3)),
            row=row, col=col)
        fig.add_trace(
            go.Scatter(x=np.hstack((Alpha , Alpha[::-1])), 
                       y=np.hstack((Error_Means[i]+Error_Stds[i], Error_Means[i]-Error_Stds[i])), 
                       fill='toself', fillcolor=PLOTLY_COLORS_LIGHT[i], opacity = 0.4+0.1*i,
                       line=dict(color=PLOTLY_COLORS_LIGHT[i]), showlegend=False),
            row=row, col=col,
        )

    fig.update_xaxes(title_text="$\\text{Level }\\alpha$", ticks="inside", tickson="boundaries", row = row, col = col)
    fig.update_yaxes(title_text="$\\left| t_m - VaR_{\\alpha}(X)\\right|$", ticks="inside", tickson="boundaries", row= row, col = col)
fig.update_layout(height=1350, width=900, title_text="$\\text{Computing VaR via UBSR Estimation (} t_m \\text{ is given by UBSR-SB Algorithm)}$.")
fig.show()

## OCE-Estimation

### Example 1: Conditional Value-at-Risk (CVaR)

In [6]:
get_u = lambda a :(lambda x : np.where(x > 0, x, 0)/(1-a), lambda x : np.where(x > 0, 1, 0)/(1-a))

def simulate_CVaR(delta, epsilon, Alpha, M, distribution):
    true_SR = -distribution.quantile(1-Alpha)
    OCE_ALL = [OCE(*get_u(a)) for a in Alpha]
    true_OCE = distribution.superquantile(Alpha)
    Error_Means_VaR, Error_Stds_VaR, Error_Means_CVaR, Error_Stds_CVaR = [], [], [], []
    
    for m in M:
        
        error_mean_var, error_std_var, error_mean_cvar, error_std_cvar = [], [], [], []
        
        for i, oce in enumerate(OCE_ALL):
            
            errors_var,errors_cvar = [], []
            for _ in range(N):
                Z = distribution.sample(m)
                t, s = oce.OCE_SAA(Z,delta/np.sqrt(m), epsilon)
                errors_var.append(t - true_SR[i])
                errors_cvar.append(s - true_OCE[i])
            
            errors_var_np = np.array(errors_var)
            errors_cvar_np = np.array(errors_cvar)
            error_mean_var.append(np.abs(errors_var_np).mean())
            error_std_var.append(errors_var_np.std())
            error_mean_cvar.append(np.abs(errors_cvar_np).mean())
            error_std_cvar.append(errors_cvar_np.std())
            
        Error_Means_VaR.append(error_mean_var)
        Error_Stds_VaR.append(error_std_var)
        Error_Means_CVaR.append(error_mean_cvar)
        Error_Stds_CVaR.append(error_std_cvar)
        
    return np.array(Error_Means_VaR), np.array(Error_Stds_VaR), np.array(Error_Means_CVaR), np.array(Error_Stds_CVaR)

In [7]:
distributions = [
    Gaussian(5,10),Gaussian(-5,10),
    Student_t(3, 3, 5), Student_t(5, -3, 4),
    Logistic(0,1), Laplace(1,1),
    GEV(1,2,0.5), GEV(1,2,-0.5)
]
subplot_titles = ['Gaussian ', 'Gaussian ',  "Student's t (df = 3)", "Student's t (df = 3)", 
                 'Logistic', 'Logistic',  'Laplace', 'Laplace',
                 'GEV (Frechet)', 'GEV (Frechet)', 'GEV (reversed Weibull)', 'GEV (reversed Weibull)']

fig = make_subplots(rows=6, cols=2, subplot_titles = subplot_titles, vertical_spacing=0.06, horizontal_spacing = 0.1)
rows = [1,2,3,4,5,6]
labels = ['m='+str(m) for m in M]
for row, distribution in zip(rows, distributions): 
    Error_Means_VaR, Error_Stds_VaR, Error_Means_CVaR, Error_Stds_CVaR = simulate_CVaR(delta, epsilon, Alpha, M, distribution)
    flag = (row == 1)
    for i in range(len(M)):
        fig.add_trace(
            go.Scatter(x=Alpha, y=Error_Means_VaR[i], line=dict(color=PLOTLY_COLORS_DARK[i]), showlegend=flag, legendgroup='one', name = labels[i]),
            row=row, col=1,
        )
        fig.add_trace(
            go.Scatter(x=np.hstack((Alpha , Alpha[::-1])), 
                       y=np.hstack((Error_Means_VaR[i]+Error_Stds_VaR[i], Error_Means_VaR[i]-Error_Stds_VaR[i])), 
                       fill='toself', fillcolor=PLOTLY_COLORS_LIGHT[i], opacity = 0.4+0.1*i,
                       line=dict(color=PLOTLY_COLORS_LIGHT[i]), showlegend=False, legendgroup='one'),
            row=row, col=1,
        )
        fig.add_trace(
            go.Scatter(x=Alpha, y=Error_Means_CVaR[i], line=dict(color=PLOTLY_COLORS_DARK[i]), showlegend=False, legendgroup='one'),
            row=row, col=2,
        )
        fig.add_trace(
            go.Scatter(x=np.hstack((Alpha , Alpha[::-1])), 
                       y=np.hstack((Error_Means_CVaR[i]+Error_Stds_CVaR[i], Error_Means_CVaR[i]-Error_Stds_CVaR[i])), 
                       fill='toself', fillcolor=PLOTLY_COLORS_LIGHT[i], opacity = 0.5,
                       line=dict(color=PLOTLY_COLORS_LIGHT[i]), showlegend=False, legendgroup='one'),
            row=row, col=2,
        )
fig.update_layout(height=1350, width=900, title_text="$\\text{Computing VaR and CVaR via OCE Estimation (} t_m, s_m \\text{ given by OCE-SB, OCE-SAA Algorithms)}$")
fig.update_xaxes(title_text="$\\text{Level }\\alpha$" , ticks="inside", tickson="boundaries")
fig.update_yaxes(title_text="$\\left| t_m - VaR_{1-\\alpha}(X)\\right|$", col = 1)
fig.update_yaxes(title_text="$\\left| s_m - CVaR_{\\alpha}(X)\\right|$", col = 2)
fig.show()